In [15]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from scipy.optimize import curve_fit
import datetime
from matplotlib.dates import date2num
import numpy as np
import plotly
import chart_studio
import chart_studio.plotly as py
from plotly.offline import plot, iplot

pacienti = pd.read_csv('pacienti_agg.csv', parse_dates=[0], dayfirst=True)
predictii = pd.read_csv('predictii_cumulat.csv', parse_dates=[0], dayfirst=True)

In [16]:
#Total cazuri
total = pacienti.merge(predictii, left_on='data', right_on='data', how='outer')
x = total['data']
y1 = total['cumulat']
y2 = total['t1']

fig = go.Figure(data=[
    go.Bar(name='Predictie', x=x, y=y2),
    go.Bar(name='Real', x=x, y=y1)
])

fig.update_layout(title="Numar total de cazuri")

fig.show()
#py.iplot(fig, filename="total_cazuri")
pio.write_html(fig, file='infectii_totale.html', auto_open=True)

In [3]:
#Cazuri externe pe zi
x = total['data']
y = total['pe_zi']

fig = go.Figure(data=[
    go.Bar(x=x, y=y)
])
fig.update_layout(title="Cazuri noi pe zi")
fig.show()

#py.iplot(fig, filename="cazuri_noi_pe_zi")
pio.write_html(fig, file='infectii_noi_zi.html', auto_open=True)

In [19]:
def sigmoid(x, a, b):
    return 1 / (1 + np.exp(a*x + b))

def logistic(x, x0,y0,c,k):
    return c / (1 + np.exp(-k*(x-x0))) + y0

def exponential(x, a, b, c, d):
    return (a * (d **(b * x))) + c

def parabolic(x, a, b, c):
    return a*x**2 + b*x + c

def plot_prediction(func, pacienti, title, filename=None, divider=1000):
    forward = 14
    l = len(pacienti)
    x = np.linspace(1, l, l)
    y = list(i/divider for i in pacienti["cumulat"])

    popt, pcov = curve_fit(func, x, y, maxfev=1000000)

    yp = pacienti['cumulat']

    x2 = np.linspace(1, l+forward, l+forward)
    date = pd.date_range(start='26-02-2020', periods=l+forward)

    fig = go.Figure(data=[
        go.Scatter(name='Predictie', x=date, y=divider*func(x2, *popt), mode='lines+markers'),
        go.Bar(name='Real', x=date, y=yp)
    ])
    fig.update_layout(title=title)
    
    if filename:
        #py.iplot(fig, filename=filename)
        pio.write_html(fig, file=filename+'.html', auto_open=True)
    else:
        fig.show()
    
def plot_predictions_history(func, pacienti, title, filename=None, divider=1000):
    forward = 14
    l = len(pacienti)
    date = pd.date_range(start='26-02-2020', periods=l+forward)
    x2 = np.linspace(1, l+forward, l+forward)

    figure_data = []
    for i in range(39, l+1):
        if divider == 10000 and i < 26:
            divider2 = 1000
        else:
            divider2 = divider
            
        workset = pacienti[:i]
        l_2 = len(workset)
        x_2 = np.linspace(1, l_2, l_2)
        y_2 = list(i/divider2 for i in workset["cumulat"])
        popt_2, pcov_2 = curve_fit(func, x_2, y_2, maxfev=10000000)
        figure_data.append(go.Scatter(name=f'{date[i].date()}', x=date, y=divider2*func(x2, *popt_2)),)

    fig = go.Figure(data=figure_data)
    fig.update_layout(title=title)
    
    if filename:
        #py.iplot(fig, filename=filename)
        pio.write_html(fig, file=filename+'.html', auto_open=True)
    else:
        fig.show()

In [20]:
plot_prediction(logistic, pacienti, "Cazuri totale + predictie curenta", divider=100000)
plot_predictions_history(logistic, pacienti, "Evolutia predictiilor", divider=100000)

plot_prediction(exponential, pacienti, "Cazuri totale + predictie pesimista", divider=100)
plot_predictions_history(exponential, pacienti, "Evolutia predictiilor pesimiste", divider=100)

plot_prediction(parabolic, pacienti, "Cazuri totale + predictie parabolica", divider=1)
plot_predictions_history(parabolic, pacienti, "Evolutia predictiilor parabolice", divider=1)

#plot_prediction(sigmoid, pacienti, "Cazuri totale + predictie sigmoid", divider=1000)
#plot_predictions_history(sigmoid, pacienti, "Evolutia predictiilor sigmoid", divider=1000)

In [6]:
plot_prediction(logistic, pacienti, "Cazuri totale + predictie curenta", "predictie_curenta", divider=100000)
plot_predictions_history(logistic, pacienti, "Evolutia predictiilor", "evolutie_predictii", divider=100000)

plot_prediction(exponential, pacienti, "Cazuri totale + predictie pesimista", "predictie_pesimista", divider=10)
plot_predictions_history(exponential, pacienti, "Evolutia predictiilor pesimiste", "evolutie_predictii_pesimiste", divider=10)

plot_prediction(parabolic, pacienti, "Cazuri totale + predictie parabolica", "predictie_parabolica", divider=1)
plot_predictions_history(parabolic, pacienti, "Evolutia predictiilor parabolice", "evolutie_predictii_parabolice", divider=1)

#plot_prediction(sigmoid, pacienti, "Cazuri totale + predictie sigmoid", "predictie_sigmoid", divider=1000)
#plot_predictions_history(sigmoid, pacienti, "Evolutia predictiilor sigmoid", "evolutie_predictii_sigmoid", divider=1000)

In [7]:
def compare_predictions(reference, pacienti, caption, filename=None, error_filename=None):
    l = len(pacienti)

    x_reference = np.linspace(1, reference, reference)
    y = list(i for i in pacienti[:reference]["cumulat"])

    popt_e, pcov = curve_fit(exponential, x_reference, y)
    #popt_s, pcov = curve_fit(sigmoid, x_reference, y)
    popt_p, pcov = curve_fit(parabolic, x_reference, y)
    popt_l, pcov = curve_fit(logistic, x_reference, [i/100000 for i in y], maxfev=100000)

    x = np.linspace(1, l, l)
    y_e = [exponential(i, *popt_e) for i in x]
    #y_s = [sigmoid(i, *popt_s) for i in x]
    y_p = [parabolic(i, *popt_p) for i in x]
    y_l = [100000*logistic(i, *popt_l) for i in x]

    yp = pacienti['cumulat']
    date = pd.date_range(start='26-02-2020', periods=len(pacienti))

    fig = go.Figure(data=[
        go.Scatter(name='Predictie realista', x=date, y=y_l, mode='lines+markers'),
        go.Scatter(name='Predictie pesimista', x=date, y=y_e, mode='lines+markers'),
        #go.Scatter(name='Predictie parabolica', x=date, y=y_p, mode='lines+markers'),
        #go.Scatter(name='Predictie sigmoid', x=date, y=y_s, mode='lines+markers'),
        go.Bar(name='Real', x=date, y=yp)
    ])
    fig.update_layout(title=caption)
    
    if filename:
        #py.iplot(fig, filename="compara_predictii")
        pio.write_html(fig, file=f'{filename}.html', auto_open=True)
    else:
        fig.show()

    error_e = (yp - y_e)*100/yp[reference:]
    #error_s = abs(yp - y_s)*100/yp[reference_count:]
    error_p = (yp - y_p)*100/yp[reference:]
    error_l = (yp - y_l)*100/yp[reference:]

    fig = go.Figure(data=[
        go.Bar(name='Predictie realista', x=date, y=error_l),
        go.Bar(name='Predictie pesimista', x=date, y=error_e),
        #go.Bar(name='Predictie parabolica', x=date, y=error_p),
        #go.Bar(name='Predictie sigmoid', x=date, y=error_s)
    ])
    fig.update_layout(title="Erori de predictie [%]")
    
    if error_filename:
        #py.iplot(fig, filename="eroare_predictii")
        pio.write_html(fig, file=f'{error_filename}.html', auto_open=True)
    else:
        fig.show()
    

In [8]:
compare_predictions(43, pacienti, 'Predictii din urma cu o saptamana (9 Aprilie) vs real')
compare_predictions(36, pacienti, 'Predictii din urma cu doua saptamani (2 Aprilie) vs real')

In [9]:
compare_predictions(43, pacienti, 'Predictii din urma cu o saptamana (9 Aprilie) vs real', 'compara_predictii', 'eroare_predictii')
compare_predictions(36, pacienti, 'Predictii din urma cu doua saptamani (2 Aprilie) vs real', 'compara_predictii_14', 'eroare_predictii_14')

In [10]:
decese = pd.read_csv('decese_agg.csv', parse_dates=[0])
x_d = decese['data']
y1_d = decese['cumulat']
#y2 = total['t1']

fig = go.Figure(data=[
    go.Bar(name='Real', x=x_d, y=y1_d)
])
fig.update_layout(title="Numar total de decese")
fig.show()

pio.write_html(fig, file='decese.html', auto_open=True)

In [11]:
x = decese['data']
y = decese['pe_zi']

fig = go.Figure(data=[
    go.Bar(x=x, y=y)
])
fig.update_layout(title="Decese noi pe zi")
fig.show()

pio.write_html(fig, file='decese_noi_zi.html', auto_open=True)

In [12]:
plot_prediction(logistic, decese, "Decese totale + predictie", divider=100000)
plot_prediction(exponential, decese, "Decese totale + predictie", divider=100000)
#100 * decese["cumulat"]
#decese["cumulat"] = decese["cumulat"].apply(lambda x: x * 100)
#decese
plot_prediction(parabolic, decese, "Decese totale + predictie", divider=10000)

In [13]:
frontieara = pd.read_csv('frontiera.csv', parse_dates=[0])
x_d = frontieara['data']
y_intrari = frontieara['intrari']
y_iesiri = frontieara['iesiri']

fig = go.Figure(data=[
    go.Bar(name='Intrari', x=x_d, y=y_intrari),
    go.Bar(name='Iesiri', x=x_d, y=y_iesiri)
])
fig.update_layout(title="Situatia la frontiera")
fig.show()

pio.write_html(fig, file='frontiera.html', auto_open=True)

In [14]:
def exponential(x, a, b, c, d):
    return (a * (d **(b * x))) + c

l = len(pacienti)
x = np.linspace(1, l, l)
y = list(i for i in pacienti["cumulat"])

popt, pcov = curve_fit(exponential, x, y, maxfev=2000000000)

fig = go.Figure(data=[
    go.Scatter(name='Predictie', x=x, y=exponential(x, *popt), mode='lines+markers'),
    go.Bar(name='Real', x=x, y=y)
])
#fig.update_layout(title=title)

fig.show()
print(popt)
print(x)
print(y)

[ 223.53495932    1.01986438 -573.86408733    1.0733668 ]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51.]
[1.0, 1.0, 3.0, 3.0, 3.0, 3.0, 4.0, 6.0, 7.0, 9.0, 13.0, 15.0, 17.0, 29.0, 47.0, 59.0, 95.0, 123.0, 139.0, 168.0, 217.0, 260.0, 277.0, 308.0, 367.0, 433.0, 576.0, 762.0, 906.0, 1029.0, 1292.0, 1452.0, 1760.0, 1952.0, 2245.0, 2460.0, 2738.0, 3183.0, 3613.0, 3864.0, 4057.0, 4417.0, 4761.0, 5202.0, 5467.0, 5990.0, 6300.0, 6633.0, 6879.0, 7216.0, 7707.0]
